## Using topg2 to Store DataFrame to Postgres
johnlichtenstein@gmail.com
<p>2020-08-18

The performance of the Pandas to_sql method when storing on Postgres can be really bad. [Naysan Saran](https://naysan.ca/2020/06/21/pandas-to-postgresql-using-psycopg2-copy_from/) has a good writeup on right and wrong ways to write tables to Postgres with psycopg2.  One method with good performnce is to use the cursor.copy_from method. The [topg2.topg2](https://github.com/johnlichtenstein/topg2) DataFrame method implemnents this, using an interface similar to the to_sql method. <p>
The topg2.topg2 method expects a unique integer column named "id" it uses to build an index, because that's a typical thing to do. <p>
The example below reads a DataFrame with a shape of (62743, 73) from a URL and a database connection json from GoogleDrive.
- The csv for the DataFrame is on s3 where anyone can read.
- The notebook is expecting to find a file named db.json in /content/drive/My Drive/secrets/. 
    - To actually run the notebook, a user would need to supply a json that connects them to a Postgres db where the user has write access.   
    - Without a connection, the notebook is for browsing. 
    
This demo is also in a [colab](https://colab.research.google.com/drive/1DyJaFpB8zPzcj0KXcDLh49xJpQMPPvlP?usp=sharing)

In [13]:
import os
from datetime import datetime as dt
import pandas as pd
import json
import requests

import psycopg2 as pg2
from io import StringIO

import topg2 # assuming installed

### Get connection to db
There needs to be a file ~/secret/db.json that connects user to Postgres. 

In [8]:
secret = os.path.join(os.path.expanduser("~/secret"), "db.json")
tD = json.load(open(secret))
conn = pg2.connect(**tD)

### Loading a DF saved previously
And some minor edits. Using a reset_index() to create a unique field named "id" that method expects.  

In [14]:
s3L = "https://datadeloro0tutorials.s3-us-west-2.amazonaws.com/sampleData/toPg2/2020.07.07_HYUNDAI.csv"
tR = pd.read_csv(s3L)
tR = tR.reset_index().rename(columns={"index": "id"}) # topg2 expects index named id
tR.assetNumber = tR.assetNumber.str.replace("=", "").str.replace('"', "") # cleaning
tR.shape

(62743, 73)

### Show topg2 creating and appending to table
Prints show timing

In [10]:
tN = "deltable"
print (dt.now().strftime("%H:%M:%S"), "starting")
print (tR[:1000].topg2(tN, conn)) # None is desired result
print (dt.now().strftime("%H:%M:%S"), "first 1000 rows")
print (tR[1000:].topg2(tN, conn))# None is desired result
print (dt.now().strftime("%H:%M:%S"), "last 61000 rows")

18:59:51 starting
None
18:59:53 first 1000 rows
None
19:00:12 last 61000 rows


### Check that it worked

In [11]:
pd.read_sql("select count(*) from public.%s" %(tN), conn)

,count
0,62743


### Cleanup
Postpone cleanup if browsing table in Postgres

In [12]:
cur = conn.cursor()
cur.execute("drop table %s" %(tN))
conn.commit()
cur.close()
conn.close()